In [1]:
import sys
import pandas as pd
from PySide6.QtWidgets import (QTextEdit, QApplication, QWidget, QVBoxLayout, QHBoxLayout, QGridLayout, QLabel, QLineEdit, QPushButton, QMessageBox, QComboBox, QTableWidget, QTableWidgetItem, QFrame, QMainWindow, QMenuBar, QTabWidget, QGridLayout)
from PySide6.QtCore import Qt
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

In [ ]:
class RetirementApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.load_data()
        self.initUI()
        self.showMaximized()

    def load_data(self):
        self.federal_tax_rates = pd.read_csv("./data/federal_tax.csv", names=["over", "up_to", "base", "perc_onexcess"], skiprows=1)
        self.ca_tax_rates = pd.read_csv("./data/ca_state_tax.csv", names=["over", "up_to", "base", "perc_onexcess"], skiprows=1)
        self.ma_tax_rate_df = pd.read_csv("./data/ma_state_tax.csv")
        self.ma_tax_rate = self.ma_tax_rate_df['Percantage'].iloc[0]
        self.social_security_df = pd.read_csv("./data/social security.csv")
        self.social_security_dict = dict(zip(self.social_security_df['age'], self.social_security_df['amount']))
        self.inflation_df = pd.read_csv("./data/inflation.csv")
        self.inflation_rates = dict(zip(self.inflation_df['Year'], self.inflation_df['Percentag'].astype(float)))

    def calculate_tax_bracket(self, rates, revenue):
        sorted_rates = rates.sort_values("over", ascending=False)
        for index, row in sorted_rates.iterrows():
            if revenue > row['over']:
                return ((revenue - row['over']) * row['perc_onexcess']) + row['base']
        return 0

    def calculate_tax(self, state, annual_revenue):
        federal_tax = self.calculate_tax_bracket(self.federal_tax_rates, annual_revenue)

        state_tax = 0
        if state == 'ca':
            state_tax = self.calculate_tax_bracket(self.ca_tax_rates, annual_revenue)
        elif state == 'ma':
            state_tax = annual_revenue * self.ma_tax_rate

        return state_tax + federal_tax
    
    def update_results_text(self, message):
        self.results_text_edit.append(message)

    def calculate_cash_flow(self, initial_savings, ira_balance, annual_expenses, annual_interest_rate_savings, annual_interest_rate_ira, retirement_age, include_social_security, state):
        if state not in ['ca', 'ma']:
            QMessageBox.warning(self, 'Invalid Input', "Please enter 'Massachusetts' or 'California'.")
            return

        social_security_amount = self.social_security_dict.get(retirement_age, 0)
        if include_social_security not in ['yes', 'no']:
            QMessageBox.warning(self, 'Invalid Input', "Please enter 'yes' or 'no'.")
            return

        current_savings = initial_savings
        current_ira_balance = ira_balance
        total_taxes_paid = 0
        year = 2023
        current_age = retirement_age
        average_life_expectancy = 79
        medium_risk_age_lower = 77
        medium_risk_age_upper = 79
        
        savings_balance_over_time = []
        ira_balance_over_time = []
        years = []
        
        social_security_added = False  # Flag to track social security addition

        self.results_table.setRowCount(0)

        while current_savings > 0 or current_ira_balance > 0:
            if year in self.inflation_rates:
                annual_expenses *= (1 + self.inflation_rates[year])

            withdrawal_amount_savings = min(annual_expenses / 2, current_savings)
            withdrawal_amount_ira = min(annual_expenses / 2, current_ira_balance)

            interest_gained_savings = current_savings * annual_interest_rate_savings
            taxable_income = interest_gained_savings + withdrawal_amount_ira

            if include_social_security == 'yes' and not social_security_added:
                tax_on_social_security = self.calculate_tax(state, social_security_amount)
                post_tax_social_security = social_security_amount - tax_on_social_security
                current_savings += post_tax_social_security
                taxable_income += social_security_amount
                social_security_added = True  # Set the flag after adding SS amount

            total_payable_tax = self.calculate_tax(state, taxable_income)
            total_taxes_paid += total_payable_tax

            current_savings += interest_gained_savings - total_payable_tax - withdrawal_amount_savings
            current_ira_balance -= withdrawal_amount_ira
            
             # Deduct taxes from the appropriate accounts
            if current_savings >= total_payable_tax:
                current_savings -= total_payable_tax
            else:
                excess_tax = total_payable_tax - current_savings
                current_savings = 0
                current_ira_balance -= excess_tax

            if year > 2058 or (current_savings + current_ira_balance) < annual_expenses:
                
                break

            row_position = self.results_table.rowCount()
            self.results_table.insertRow(row_position)
            self.results_table.setItem(row_position, 0, QTableWidgetItem(str(year)))
            self.results_table.setItem(row_position, 1, QTableWidgetItem(f"${current_savings:,.2f}"))
            self.results_table.setItem(row_position, 2, QTableWidgetItem(f"${current_ira_balance:,.2f}"))
            self.results_table.setItem(row_position, 3, QTableWidgetItem(f"${total_payable_tax:,.2f}"))
            
            savings_balance_over_time.append(current_savings)
            ira_balance_over_time.append(current_ira_balance)
            years.append(year)

            year += 1
            current_age += 1  # Increment age each year

        # After loop, display results and plot
        final_age_when_funds_deplete = current_age
        self.plot_graph(years, savings_balance_over_time, ira_balance_over_time)
        self.update_results_text(f"Insufficient funds to cover expenses in year {year}.")
        self.update_results_text(f"Total Taxes Paid For The Whole Period: ${total_taxes_paid:.2f}.")

        if final_age_when_funds_deplete > average_life_expectancy:
            risk_level = "Low Risk"
        elif 77 <= final_age_when_funds_deplete <= 79:
            risk_level = "Medium Risk"
        else:
            risk_level = "High Risk"

        risk_message = (f"Based on the latest data from 2023, the average life expectancy in the USA is {average_life_expectancy} years. "
                        f"Your funds are expected to last until age {final_age_when_funds_deplete}, which is considered {risk_level}.")

        self.update_results_text(risk_message)

        if risk_level == "High Risk":
            advice_message = ("\nYour financial analysis indicates a High Risk of depleting your funds prematurely. Consider the following strategies to enhance your financial stability:\n"
                              "- Consider extending your working years "
                              "- Increase your retirement account contributions to maximize savings and tax benefits.\n"
                              "- Delay claiming Social Security benefits to increase monthly payments.\n"
                              "- Reassess and possibly rebalance your investment portfolio to better align with your retirement goals.\n"
                              "- Consider downsizing your living arrangements to reduce ongoing expenses.\n"
                              "- Explore part-time work or consulting to supplement your income.\n"
                              "- Look into purchasing annuities for a guaranteed income stream.\n"
                              "- Consult with a financial advisor to tailor a retirement plan that fits your unique needs.")
            self.update_results_text(advice_message)
            
        if risk_level == "Medium Risk":
            advice_message = ("\nYour financial outlook is assessed as Medium Risk, suggesting a balanced approach is needed. Here are some strategies to consider for maintaining and possibly enhancing your financial health:\n"
                              "- Regularly review your investment portfolio to ensure it matches your risk tolerance and retirement timeline.\n"
                              "- Build or maintain an emergency fund to avoid unexpected withdrawals from retirement savings.\n"
                              "- Continue enhancing your financial literacy with current information on investments and retirement strategies.\n"
                              "- Manage debts effectively to reduce interest burdens and increase available savings.\n"
                              "- Explore diversifying your income sources to reduce dependency on any single financial stream.\n"
                              "- Consider making moderate lifestyle adjustments to ensure your funds last longer.\n"
                              "- Stay informed about any policy changes that could impact your retirement plans.")
            self.update_results_text(advice_message)
            
        if risk_level == "Low Risk":
            advice_message = ("\nCongratulations! Your financial outlook is assessed as Low Risk. This suggests that your retirement planning is on a solid foundation. Here are some suggestions to further enhance your financial security and retirement experience:\n"
                              "- Continue investing wisely. Even though your current plan is effective, always look for ways to optimize your investment returns.\n"
                              "- Increase your savings rate. If possible, boost your savings to maximize your financial buffer and explore additional investment opportunities.\n"
                              "- Plan for healthcare. Ensure that you have a robust plan for healthcare costs, which can be a significant expense in retirement.\n"
                              "- Consider legacy planning. Think about how you want to manage your estate and any legacy you wish to leave for your family or charities.\n"
                              "- Enjoy your achievements. Consider how you can use your well-managed funds to enjoy life more, perhaps by traveling, pursuing hobbies, or helping others.\n"
                              "- Stay informed about financial trends and new retirement planning tools and strategies to continuously refine your approach.")
            self.update_results_text(advice_message)

    def plot_graph(self, years, savings_balance_over_time, ira_balance_over_time):
        self.figure.clear()
        ax = self.figure.add_subplot(111)
        ax.plot(years, savings_balance_over_time, label='Savings Balance', marker='o', linestyle='-')
        ax.plot(years, ira_balance_over_time, label='IRA Balance', marker='o', linestyle='-')
        ax.set_xlabel('Year')
        ax.set_ylabel('Amount ($)')
        ax.set_title('Savings and IRA Balance Over Time')
        ax.legend()
        ax.grid(True)
        self.canvas.draw()

    def __init__(self):
        super().__init__()
        self.load_data()
        self.setupWidgets()
        self.initUI()
        self.showMaximized()
        
    def initAllWidgets(self):
        self.layout = QVBoxLayout(self)
        self.input = QLineEdit("Hover over me", self)
        self.input.setToolTip("This is a tooltip example.")
        self.layout.addWidget(self.input)
        
    def setupWidgets(self):
        self.initial_savings_input = QLineEdit()
        self.initial_savings_input.setPlaceholderText("Hover over me for additional instructions")
        self.ira_balance_input = QLineEdit()
        self.annual_expenses_input = QLineEdit()
        self.interest_rate_savings_input = QLineEdit()
        self.interest_rate_ira_input = QLineEdit()
        self.retirement_age_input = QLineEdit()
        self.include_social_security_input = QComboBox()
        self.include_social_security_input.addItems(['yes', 'no'])
        self.state_input = QComboBox()
        self.state_input.addItems(['Massachusetts', 'California'])
        self.calculate_button = QPushButton('Calculate Cash Flow')
        self.calculate_button.clicked.connect(self.on_calculate_button_clicked)
        
        self.initial_savings_input.setToolTip("Enter the initial amount of savings you have.")
        self.ira_balance_input.setToolTip("Enter the current balance in your IRA account.")
        self.annual_expenses_input.setToolTip("Enter your expected annual expenses during retirement.")
        self.interest_rate_savings_input.setToolTip("Enter the interest rate for your savings account (in %). Ex. if you have 2% interest you shoud enter: 0.02.")
        self.interest_rate_ira_input.setToolTip("Enter the interest rate for your IRA (in %).Ex. if you have 2% interest you shoud enter: 0.02")
        self.retirement_age_input.setToolTip("Enter the age at which you plan to retire.")
        self.include_social_security_input.setToolTip(
        "Select 'yes' if you wish to include social security benefits.""Depending on the selected retirement age, a corresponding social security benefit "
        "will be added to your initial savings. This benefit varies by age:\n"
        "- Age 62: $25,908\n"
        "- Age 63: $27,780\n"
        "- Age 64: $29,640\n"
        "- Age 65: $32,124\n"
        "- Age 66: $34,608\n"
        "- Age 67: $37,104\n"
        "- Age 68: $40,092\n"
        "- Age 69: $43,080\n"
        "- Age 70: $46,080\n"
        "This one-time addition boosts your funds upon retirement.")
        self.state_input.setToolTip("Select the state you live in to consider local tax implications.")

    def initUI(self):
         # Create a central widget and set the main layout on it
        central_widget = QWidget(self)  # Ensure the central widget is set
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget) 
        
            # Create the tab widget
        tab_widget = QTabWidget()
        main_layout.addWidget(tab_widget)

        # Setup the first tab
        tab1 = QWidget()
        tab1_layout = QVBoxLayout(tab1)
        self.setupTab1(tab1_layout)
        tab_widget.addTab(tab1, "Retirement Simulation and Interactive Analysis")
        
            # Setup the second tab for displaying dynamic results
        tab2 = QWidget()
        tab2_layout = QVBoxLayout(tab2)
        self.results_text_edit = QTextEdit()
        self.results_text_edit.setReadOnly(True)  # Make the QTextEdit read-only
        self.results_text_edit.setStyleSheet("font-size: 14px; padding: 10px; background-color: #f0f0f0;")
        tab2_layout.addWidget(self.results_text_edit)
        tab_widget.addTab(tab2, "Financial Projections and Planning Outcomes")
        
 #main_layout = QVBoxLayout()  # Main layout to hold top and bottom sections
        self.setStyleSheet("""
        QWidget {
            font-size: 13px;
        }
        QLabel {
            font-weight: bold;
            margin-top: 5px;
        }
        QLineEdit, QComboBox {
            margin-bottom: 5px;
            padding: 5px;
            font-size: 13px;
        }
        QPushButton {
            padding: 7px;
            background-color: #5cb85c;
            color: white;
            font-weight: bold;
        }
        QTableWidget {
            margin-top: 7px;
            font-size: 13px;
            selection-background-color: #c0c0c0;
            selection-color: black;
        }
        QHeaderView::section {
            background-color: #5cb85c;
            color: white;
            padding: 4px;
            border: 1px solid #d0d0d0;
        }
        """)
        

        
        # Menu Bar setup
        menu_bar = self.menuBar()
        file_menu = menu_bar.addMenu("File")
        download_action = file_menu.addAction("Download")
        reset_action = menu_bar.addAction("Reset")
        reset_action.triggered.connect(self.reset_app)
        download_action.triggered.connect(self.download_data)
        
            # Create an 'About' menu
        about_menu = menu_bar.addMenu("About")
        about_calculator_action = about_menu.addAction("About Calculator")
        about_calculator_action.triggered.connect(self.about_calculator)

        # Create an 'Information' menu
        about_developer_action = about_menu.addAction("About the Developer")
        about_developer_action.triggered.connect(self.creator_info)
        
         # Grid layout for labels and fields
        input_layout = QGridLayout()  

        # Add a heading label
        heading_label = QLabel("Input Parameters")
        heading_label.setStyleSheet("font-size: 16px; font-weight: bold; padding: 10px;")  # Styling the heading
        input_layout.addWidget(heading_label, 1, 1, 1, 2) 

        
        
    def on_calculate_button_clicked(self):
        try:
            # Debug print statements to check what is being captured
            # Validate and convert input values
            initial_savings = self.initial_savings_input.text().strip()
            if not initial_savings:
                raise ValueError("Initial savings is required.")
            initial_savings = float(initial_savings)

            ira_balance = self.ira_balance_input.text().strip()
            if not ira_balance:
                raise ValueError("IRA balance is required.")
            ira_balance = float(ira_balance)

            annual_expenses = self.annual_expenses_input.text().strip()
            if not annual_expenses:
                raise ValueError("Annual expenses is required.")
            annual_expenses = float(annual_expenses)

            interest_rate_savings = self.interest_rate_savings_input.text().strip()
            if not interest_rate_savings:
                raise ValueError("Interest rate for savings is required.")
            interest_rate_savings = float(interest_rate_savings)

            interest_rate_ira = self.interest_rate_ira_input.text().strip()
            if not interest_rate_ira:
                raise ValueError("Interest rate for IRA is required.")
            interest_rate_ira = float(interest_rate_ira)

            retirement_age = self.retirement_age_input.text().strip()
            if not retirement_age:
                raise ValueError("Retirement age is required.")
            retirement_age = int(retirement_age)

            include_social_security = self.include_social_security_input.currentText()
            state = self.state_input.currentText().lower()[:2]

            self.calculate_cash_flow(initial_savings, ira_balance, annual_expenses, interest_rate_savings, interest_rate_ira, retirement_age, include_social_security, state)

        except ValueError as e:
            QMessageBox.warning(self, 'Input Error', str(e))
        
    def setupTab1(self, tab1_layout):
        # Input and Button Layout
        input_layout = QGridLayout()  # Grid for input labels and fields
        labels = ['Initial Savings:', 'IRA Balance:', 'Annual Expenses:', 'Interest Rate for Savings:', 
                  'Interest Rate for IRA:', 'Retirement Age:', 'Include Social Security:', 'State:']
        tooltips = [
            "Enter the amount of savings you currently have available for retirement.",
            "Enter the balance of your Individual Retirement Account (IRA).",
            "Estimate the total amount of expenses you expect to have annually after retiring.",
            "Enter the interest rate for your savings account.",
            "Enter the interest rate for your IRA investments.",
            "Enter the age at which you plan to retire.",
            "Choose whether to include social security benefits in your retirement plan.",
            "Select your state to consider specific tax implications."
        ]
        widgets = [self.initial_savings_input, self.ira_balance_input, self.annual_expenses_input,
                   self.interest_rate_savings_input, self.interest_rate_ira_input, self.retirement_age_input,
                   self.include_social_security_input, self.state_input]

        for i, (label, widget) in enumerate(zip(labels, widgets)):
            label_widget = QLabel(label)
            input_layout.addWidget(label_widget, i, 0)
            input_layout.addWidget(widget, i, 1)
            print(f"Added {label} widget to layout.")

        input_layout.addWidget(self.calculate_button, len(labels), 0, 1, 2)  # Span two columns

        # Results Table
        self.configureResultsTable()
        
        # Top Horizontal Layout
        top_layout = QHBoxLayout()
        top_layout.addLayout(input_layout, 1)  # Input fields on the right
        top_layout.addWidget(self.results_table, 1)  # Results table on the left
        
         # Add the top layout to the tab's main layout
        tab1_layout.addLayout(top_layout, 1)

        self.figure = Figure(figsize=(8, 4))
        self.canvas = FigureCanvas(self.figure)
        bottom_layout = QVBoxLayout()
        bottom_layout.addWidget(self.canvas)

        # Graph Layout
        bottom_layout = QVBoxLayout()
        bottom_layout.addWidget(self.canvas, 2)  # Assign a higher stretch factor to give more space

        # Combine top (inputs and table) and bottom (graph) layouts into the tab's layout
        tab1_layout.addLayout(bottom_layout, 2)  # Adding more space to the bottom layout where the graph is

        self.plot_graph([], [], [])  

        self.setWindowTitle('Retirement Cash Flow Calculator')
        
    def update_results_text(self, message):
        """Appends the provided message to the results text edit widget on the second tab."""
        self.results_text_edit.append(message)  # Appends new message
        self.results_text_edit.append("\n")  # Adds a newline for separation between messages


    def configureResultsTable(self):
        self.results_table = QTableWidget()
        self.results_table.setColumnCount(4)
        self.results_table.setHorizontalHeaderLabels(["Year", "Savings Balance", "IRA Balance", "Taxes Paid"])
        self.results_table.setColumnWidth(0, 80)
        self.results_table.setColumnWidth(1, 140)
        self.results_table.setColumnWidth(2, 140)
        self.results_table.setColumnWidth(3, 100)
        self.results_table.horizontalHeader().setStretchLastSection(True)
        self.results_table.setAlternatingRowColors(True)
        self.results_table.setSortingEnabled(True)
         
        
    def reset_app(self):
        # Clear all input fields and reset selections
        self.initial_savings_input.clear()
        self.ira_balance_input.clear()
        self.annual_expenses_input.clear()
        self.interest_rate_savings_input.clear()
        self.interest_rate_ira_input.clear()
        self.retirement_age_input.clear()
        self.include_social_security_input.setCurrentIndex(0)  # Assuming 'yes' is at index 0
        self.state_input.setCurrentIndex(0)  # Assuming 'Massachusetts' is at index 0
        self.results_table.setRowCount(0)
        self.figure.clear()
        self.canvas.draw()
        self.results_text_edit.clear()
        
    def about_calculator(self):
        message = """
        <h1>About the Retirement Calculator</h1>
        <p>This calculator helps users estimate how long their retirement savings might last, factoring in initial savings, IRA balances, annual expenses, and applicable interest rates. It is designed to provide a tax-efficient withdrawal strategy by splitting annual expenses between the savings and IRA accounts.</p>
        <p>The calculator adjusts for inflation annually and considers different state tax regimes for California and Massachusetts. It also incorporates social security benefits if applicable, which are added to the savings after taxing them.</p>
        <p>By minimizing taxable withdrawals from the IRA and optimizing the use of savings, the calculator aims to extend the lifespan of the retiree’s funds. It offers a detailed annual breakdown of expenses, taxes, and remaining balances to help users plan their retirement effectively.</p>
        <p>Tax calculations include both state and federal taxes based on the income generated from the IRA and interest earned from savings accounts. The system is structured to help retirees manage their cash flows efficiently, maintaining sufficient funds to cover expenses while minimizing the tax impact.</p>
        """
        QMessageBox.information(self, "About Calculator", message)

    def creator_info(self):
        message = """
        <h1>About the Developer</h1>
        <p>This application was developed by Alexander Yankov, who is a recent graduate in Master's degree in Business Analytics. As his first project, this Retirement Calculator aims to establish his portfolio and showcase his analytical capabilities through practical application, intended for public use on the internet. Connect with Alexander Yankov on LinkedIn to explore his professional profile and other projects:</p>
        <p>For more information, please contact: ayankov@outlook.com </p>
        """
        QMessageBox.information(self, "About the Developer", message)
        
    def download_data(self):
        # Placeholder for download functionality
        # This example simply saves the data to a CSV file, modify as needed
        file_path = QFileDialog.getSaveFileName(self, "Save File", "", "CSV Files (*.csv);;All Files (*)")
        if file_ipk_path[0]:
            try:
                # Assume you have a DataFrame `df` you want to save
                # You'll need to adapt this to your application's needs
                self.data_frame.to_csv(file_path[0], index=False)
                QMessageBox.information(self, "Download Complete", "Data has been saved successfully.")
            except Exception as e:
                QMessageBox.warning(self, "Download Failed", f"Failed to save the file.\n{str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = RetirementApp()
    ex.show()
    sys.exit(app.exec())

Added Initial Savings: widget to layout.
Added IRA Balance: widget to layout.
Added Annual Expenses: widget to layout.
Added Interest Rate for Savings: widget to layout.
Added Interest Rate for IRA: widget to layout.
Added Retirement Age: widget to layout.
Added Include Social Security: widget to layout.
Added State: widget to layout.
